# Example Network

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Generate synthetic dataset
X, y = make_moons(n_samples=1000, noise=0.2, random_state=0)
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Define simple NN
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.net(x)

model = SimpleNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(300):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    preds = (model(X_test) > 0.5).float()
    accuracy = (preds.eq(y_test).sum() / len(y_test)).item()
print(f'Test Accuracy: {accuracy:.4f}')


Test Accuracy: 0.9100


# Define Attack Setup

In [2]:
from framework.attack import injector
from framework.criterion import binary_classifcation_accuracy
import pandas as pd

inj = injector(model, X_test, y_test, binary_classifcation_accuracy)

results = inj.run_seu(0)

pd.DataFrame(results)

Testing a forward pass on cuda...
Basline Criterion Score: 0.91


,tensor_location,criterion_score,layer_name,value_before,value_after
0,"(0, 0)",0.830000,net.0.weight,2.130202,-2.130202
1,"(0, 1)",0.920000,net.0.weight,0.544638,-0.544638
2,"(1, 0)",0.916667,net.0.weight,-0.129522,0.129522
3,"(1, 1)",0.933333,net.0.weight,0.853291,-0.853291
4,"(2, 0)",0.856667,net.0.weight,-1.868104,1.868104
5,"(2, 1)",0.916667,net.0.weight,0.539121,-0.539121
6,"(3, 0)",0.913333,net.0.weight,0.216950,-0.216950
7,"(3, 1)",0.916667,net.0.weight,-0.409378,0.409378
8,"(4, 0)",0.910000,net.0.weight,-0.344291,0.344291
9,"(4, 1)",0.910000,net.0.weight,-0.042809,0.042809
